### Import Libraries and modules

In [12]:
import pandas as pd
import numpy as np
import requests
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

Get data from Coingecko API

In [21]:
def get_token_data(token_ids):
    url = "https://api.coingecko.com/api/v3/coins/markets"
    params = {
        "vs_currency": "usd",
        "ids": ",".join(token_ids),
        "price_change_percentage": "1h,24h,7d,30d,1y"
    }
    response = requests.get(url, params=params)
    data = response.json()
    df = pd.DataFrame(data)
    
    cols = ["id", "symbol", "current_price", "ath", "market_cap_rank", "price_change_percentage_30d_in_currency", "price_change_percentage_1y_in_currency"]
    cols = [c for c in cols if c in df.columns] 
    df = df[cols]
    df.rename(columns={"price_change_percentage_30d_in_currency": "price_change_30d", "price_change_percentage_1y_in_currency" : "price_change_1y"}, inplace=True)
    return df


tokens = ["bitcoin", "ethereum", "solana", "cardano", "dogecoin", "shiba-inu", "pepe", "kadena"]

df = get_token_data(tokens)
print("\n📊 Raw data:\n", df.head())



📊 Raw data:
          id symbol  current_price            ath  market_cap_rank  \
0   bitcoin    btc  111415.000000  126080.000000                1   
1  ethereum    eth    3937.740000    4946.050000                2   
2    solana    sol     192.190000     293.310000                6   
3  dogecoin   doge       0.197000       0.731578                9   
4   cardano    ada       0.654684       3.090000               11   

   price_change_30d  price_change_1y  
0          0.376418        64.612854  
1         -0.074186        56.145112  
2         -4.398170        11.758674  
3        -14.656241        44.908596  
4        -17.190148        91.488470  


Create features

In [22]:
df["price_drop_from_ath"] = (df["ath"] - df["current_price"]) / df["ath"] * 100
df["is_strong"] = (df["price_change_30d"] > 0).astype(int)

features = df[["price_drop_from_ath", "price_change_30d", "price_change_1y"]]
labels = df["is_strong"]

Train Model

In [23]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.3, random_state=42)

model = RandomForestClassifier(n_estimators=50, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print("✅ Model evaluation:\n")
print(classification_report(
    y_test,
    y_pred,
    labels=[0, 1],
    target_names=["Weak", "Strong"],
    zero_division=0
))



✅ Model evaluation:

              precision    recall  f1-score   support

        Weak       0.67      1.00      0.80         2
      Strong       0.00      0.00      0.00         1

    accuracy                           0.67         3
   macro avg       0.33      0.50      0.40         3
weighted avg       0.44      0.67      0.53         3



Predict

In [19]:
proba = model.predict_proba(features)

if proba.shape[1] == 1:
    # Only one class present — just use model.predict() instead
    df["strength_score"] = proba[:, 0]
    df["Health"] = np.where(df["strength_score"] > 0.5, "🟢 Strong", "🔴 Weak")
else:
    # Normal case: two-class probability
    df["strength_score"] = proba[:, 1]
    df["Health"] = np.where(df["strength_score"] > 0.5, "🟢 Strong", "🔴 Weak")

print("\n🌐 Token Strength Report:\n")
print(df[["symbol", "current_price", "price_drop_from_ath", "price_change_30d", "Health"]])



🌐 Token Strength Report:

  symbol  current_price  price_drop_from_ath  price_change_30d    Health
0    btc  111255.000000            11.758407         -0.267333  🟢 Strong
1    eth    3932.800000            20.486044         -1.947774  🟢 Strong
2    sol     191.730000            34.632300         -5.435645  🟢 Strong
3   doge       0.196325            73.164174        -14.948961  🟢 Strong
4    ada       0.653257            78.858997        -17.784660  🟢 Strong
5   shib       0.000010            88.231198        -15.006210  🟢 Strong
6   pepe       0.000007            74.527292        -24.464454  🟢 Strong
